In [2]:
import pandas as pd

In [11]:
game_metadata = pd.read_csv('./out/game_metadata.csv')
game_player_stats = pd.read_csv('./out/game_player_stats.csv')

unique_games = game_metadata['gameid'].size
print(f'Amount of unique games: {unique_games}')
print(50*'-')

print('Types of values in each dataframe:')
print('Game_metadata:')
print(game_metadata.dtypes)

print('')

print('game_player_stats')
print(game_player_stats.dtypes)
print(50*'-')

Amount of unique games: 74105
--------------------------------------------------
Types of values in each dataframe:
Game_metadata:
gameid         object
date           object
league         object
playoffs        int64
patch         float64
gamelength      int64
dtype: object

game_player_stats
gameid                object
playerid              object
teamid                object
side                  object
position              object
champion              object
result                 int64
kills                  int64
deaths                 int64
assists                int64
damagetochampions    float64
visionscore          float64
earnedgold           float64
total cs             float64
golddiffat15         float64
csdiffat15           float64
xpdiffat15           float64
dtype: object
--------------------------------------------------
